# Dog-Cat Classification
21/09/2022 by LimBus

**Pipeline**
*   Get & Prepare dataset
*   Build the model
*   Train the model
*   Evaluate the trained model

## Get & Prepare dataset

In [ ]:
# import libraries

import os
import zipfile
import matplotlib.image as mpimg

print('cell complete!')

In [ ]:
# load dataset

!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

print('cell complete!')

In [ ]:
# unzip

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

print('cell complete!')

In [ ]:
# Set directory path

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

print('cell complete!')

In [ ]:
# get list of file names

train_cat_fnames = os.listdir(train_cats_dir)
print(train_cat_fnames[:10])

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

print('cell complete!')

In [ ]:
# print total lenght of train/validation set

print('total training cat images: {}'.format(len(train_cat_fnames)))
print('total training dog images: {}'.format(len(train_dog_fnames)))
print('total validation cat images: {}'.format(len(os.listdir(validation_cats_dir))))
print('total validation dog images: {}'.format(len(os.listdir(validation_dogs_dir))))

print('cell complete!')

In [ ]:
# data visualization
import matplotlib.pyplot as plt

nrows = 4
ncols = 4

pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_cat_pix = [os.path.join(train_cats_dir, fname)
                for fname in train_cat_fnames[pic_index-8:pic_index]]
next_dog_pix = [os.path.join(train_dogs_dir, fname)
                for fname in train_dog_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  sp = plt.subplot(nrows, ncols, i+1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

print('cell complete!')

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms

#transformations
train_transforms = transforms.Compose([transforms.Resize((225,225)),
                                       transforms.ToTensor(),                               
                                       torchvision.transforms.Normalize(
                                           mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225],),
                                       ])
test_transforms = transforms.Compose([transforms.Resize((225,225)),
                                      transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],),
                                      ])

#datasets
train_data = datasets.ImageFolder(train_dir,transform=train_transforms)
test_data = datasets.ImageFolder(validation_dir,transform=test_transforms)

#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=20)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=20)

In [ ]:
import matplotlib.pyplot as plt
from torch.autograd import Variable

examples = enumerate(trainloader)
print("len: ", len(trainloader))
idx, batch = next(examples)
inp_0 = Variable(batch[0][0]).squeeze()
print("sample shape: ", inp_0.size())
f, axr = plt.subplots(1,1)
axr.imshow(transforms.ToPILImage()(inp_0))
print('cell complete!')

## Build the model

In [ ]:
torch.cuda.is_available()

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.vgg16(pretrained=True)

#freeze all params
for params in model.parameters():
  params.requires_grad = False

#add a new final layer
nr_filters = model.classifier[0].in_features  #number of input features of last layer
model.classifier = nn.Sequential(
    nn.Linear(nr_filters, 1, bias=False))
model.classifier[0].weight.data.zero_()
model = model.to(device)
model

In [ ]:
examples = enumerate(trainloader)
print("len: ", len(trainloader))
idx, batch = next(examples)
inp_0 = Variable(batch[0][0]).squeeze()
print("sample shape: ", inp_0.size())

In [ ]:
from torchsummary import summary
summary(model, inp_0.shape)

## Train the model

In [ ]:
from torch.nn.modules.loss import BCEWithLogitsLoss

# loss
loss_fn = BCEWithLogitsLoss() # binary cross entropy with sigmoid, so no need to use sigmoid in the model

# optimizer
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=.001) 
optimizer

In [ ]:
# hyperparameters setting
losses = []
val_losses = []

epoch_train_losses = []
epoch_test_losses = []

n_epochs = 10
early_stopping_tolerance = 3
early_stopping_threshold = 0.03

In [ ]:
from tqdm import tqdm

for epoch in range(n_epochs):
  epoch_loss = 0
  for i ,data in tqdm(enumerate(trainloader), total = len(trainloader)): # iterate ove batches
    x_batch , y_batch = data
    x_batch = x_batch.to(device) # move to gpu
    y_batch = y_batch.unsqueeze(1).float() # convert target to same nn output shape
    y_batch = y_batch.to(device) # move to gpu

    # enter train mode
    model.train()

    # make prediction
    yhat = model(x_batch)

    # compute loss
    loss = loss_fn(yhat,y_batch)

    # conpute grad
    optimizer.zero_grad()
    loss.backward()

    # update parameters
    optimizer.step()

    epoch_loss += loss/len(trainloader)
    losses.append(loss)
    
  epoch_train_losses.append(epoch_loss)
  print('Epoch : {}, train loss : {}'.format(epoch+1,epoch_loss))

  # validation doesnt requires gradient
  with torch.no_grad():
    cum_loss = 0
    for x_batch, y_batch in testloader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float() # convert target to same nn output shape
      y_batch = y_batch.to(device)

      # model to eval mode
      model.eval()

      yhat = model(x_batch)
      val_loss = loss_fn(yhat,y_batch)
      cum_loss += val_loss/len(testloader)
      val_losses.append(val_loss.item())


    epoch_test_losses.append(cum_loss)
    print('Epoch : {}, val loss : {}'.format(epoch+1,cum_loss))  
    
    best_loss = min(epoch_test_losses)
    print('Epoch : {}, best loss : {}'.format(epoch+1,best_loss))
    
    # save best model
    if cum_loss <= best_loss:
      best_model_wts = model.state_dict()
    
    # early stopping
    early_stopping_counter = 0
    if cum_loss > best_loss:
      early_stopping_counter +=1

    if (early_stopping_counter == early_stopping_tolerance) or (best_loss <= early_stopping_threshold):
      print("Terminating: early stopping")
      break # terminate training

In [ ]:
#load best model
model.load_state_dict(best_model_wts)
checkpoint_dict = {
    'model_state_dict' : model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict()}

In [ ]:
torch.save(checkpoint_dict, '/content/mymodel.pth')

## Evaluate the trained model

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
  
unorm = UnNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms

def inference(test_data):
  idx = torch.randint(1, len(test_data), (1,))
  sample = torch.unsqueeze(test_data[idx][0], dim=0).to(device)

  p = torch.sigmoid(model(sample))
  p = p.to('cpu').data.item()
  if p < 0.5:
    print(f"Prediction : Cat {1-p} & Dog {p}")
  else:
    print(f"Prediction : Dog {p} & Cat {1-p}")

  plt.imshow(transforms.ToPILImage()(unorm(test_data[idx][0])))

inference(test_data)